# Text Pre Processor - Experimento

Este é um componente que utiliza a biblioteca [nltk](https://www.nltk.org/) e [ftfy](https://pypi.org/project/ftfy/) e [regex](https://docs.python.org/3/library/re.html) para pré processar textos que entrrão em outros componentes.

Este notebook apresenta:
- como usar o [SDK](https://platiagro.github.io/sdk/) para carregar datasets, salvar modelos e outros artefatos.
- como declarar parâmetros e usá-los para criar componentes reutilizáveis.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
O parâmetro `dataset` identifica os conjuntos de dados. Você pode importar arquivos de dataset com o botão  na barra de ferramentas.

In [1]:
# parâmetros
dataset = "/tmp/data/imdb-2.csv" #@param {type:"string"}
target = "sentiment" #@param {type:"string", label:"Atributo alvo", description:"Seu modelo será treinado para prever os valores do alvo."}
language = "english" #@param ["portuguese", "english"] {type:"string", label:"Linguagem", description:"Linguagem da qual os stopwords pertencem. Deve ser a mesma utilizada no dataset."}

# selected features to perform the model
filter_type = "incluir" #@param ["incluir","remover"] {type:"string",label:"Modo de seleção das features", description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção [incluir]. Caso deseje informar as features que não devem ser utilizadas, selecione [remover]."}
#model_features = "review" #@param {type:"string",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}
model_features = "review" #@param {type:"string"}

# preprocessamento
case = "Lower" #@param ["Lower","Upper","NotApply"] {type:"string",label:"Aplicação de casing", description:"Caixa baixa, caixa alta ou não aplicação de caixa"}
remove_stop_words = True #@param {type:"boolean",label:"Remoção de Stop Words", description:"Remoção de palavras, conjunções, artigos e outros"}
remove_top_words = True #@param {type:"boolean",label:"Remoção de Top Words", description:"Remoção dea porcentagem palavras mais frequentes no texto"}
top_words_percentage = 0.01 #@param {type:"number",label:"Porcentagem de Top Words", description:"Porcentagem das palavras mais frequentes no texto"}
stemming = False #@param {type:"boolean",label:"Stemming"}
lemmatization = True #@param {type:"boolean",label:"Lemmatization"}
remove_punctuation = True #@param {type:"boolean",label:"Remoção de pontuação"}
remove_line_braks = True #@param {type:"boolean",label:"Remoção de quebras de lina",description:"Remoção de quebras de linha por \n e \r"}
remove_accents = True #@param {type:"boolean",label:"Remoção de acentos"}
remove_html = True #@param {type:"boolean",label:"Remoção de HTML"}
remove_css = True #@param {type:"boolean",label:"Remoção de CSS"}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [2]:
import pandas as pd

df = pd.read_csv(dataset)
print(df)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [3]:
import numpy as np
#from platiagro import stat_dataset

#metadata = stat_dataset(name=dataset)
#featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
#featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
#featuretypes = np.delete(featuretypes, target_index)

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [4]:
df.dropna(subset = [target],inplace=True)
df.dropna(subset = [model_features],inplace=True)
y = df[target].to_numpy()

## Filtragem das features 

Seleciona apenas as features que foram declaradas no parâmetro model_features. Se nenhuma feature for especificada, todo o conjunto de dados será utilizado para a modelagem.

In [5]:
if filter_type == 'incluir':
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        #featuretypes = featuretypes[columns_index]
    else:
        columns_to_filter = columns
else:
    if len(model_features) >= 1:
        columns_index = (np.where(np.isin(columns,model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns,columns_index)
        #featuretypes = np.delete(featuretypes,columns_index)
    else:
        columns_to_filter = columns

# keep the features selected
df_model = df[columns_to_filter]
X = df_model.to_numpy()

## Codifica labels do atributo alvo

As labels do atributo alvo são convertidos em números inteiros ordinais com valor entre 0 e n_classes-1.

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## Definição dos filtros

In [7]:
import string
from collections import defaultdict
from functools import reduce
from re import sub

import unidecode
from ftfy import fix_text
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize


def tokenize_text(text_list: list = None):
    """Tokenize Text without the hyperparâmeters defined.

    Args:
        text_list (list): a list of texts to be used.

    Returns:
        A list of tokenized text without punctuation.
    """

    tokenize_list = list()
    for text in text_list:
        text = text[0]
        text = fix_text(text)
        text = sub("<.*?>", " ", text) if remove_html else text
        text = sub("{.*?}", " ", text) if remove_css else text
        text = unidecode.unidecode(text) if remove_accents else text
        text = sub("/\r\n|\n|\r|", "", text) if remove_line_braks else text
        text = (
            sub("[" + string.punctuation + "]", "", text)
            if remove_punctuation
            else text
        )
        text = sub(" +", " ", text)  # only to avoid multiple spaces
        text = text.split(" ")

        tokenize_list.append(text)
    return tokenize_list


def top_tokens_stopwords(sentence_list: list, percentage: float = 0.01):
    """Selects the most relevant stops words of the tokerized texts.

    Args:
        sentence_list (list): list of tokens.
        percentage (float): percentage threshold.
    """
    percentage = top_words_percentage
    vocabulary = defaultdict(int)

    for sample in sentence_list:
        for token in sample:
            vocabulary[token] += 1

    all_tokens = sorted(vocabulary.items(), key=lambda token: token[1], reverse=True)
    top_tokens = all_tokens[: int(len(all_tokens) * percentage)]

    return [token[0] for token in top_tokens]


def remove_specific_tokens(sentence_list: list, tokens_to_be_removed: list = None):
    """Removes specific tokens from a token list.

    Args:
        sentence_list (list): list of tokens from which other tokens will be removed.
        tokens_to_be_removed (list): list of tokens that need to be removed.
    """
    sentence_list_ = list()
    sentence_list_ = [x for x in sentence_list if x not in tokens_to_be_removed]

    return sentence_list_


def apply_stemming(sentence_list: list):
    ps = PorterStemmer()
    sentence_list = [
        [ps.stem(word) for word in token_list] for token_list in sentence_list
    ]
    return sentence_list


def apply_lemmatization(sentence_list: list):
    lemmatizer = WordNetLemmatizer()
    sentence_list = [
        [lemmatizer.lemmatize(word) for word in token_list]
        for token_list in sentence_list
    ]
    return sentence_list


def apply_casing(sentence_list: list, case: str):
    if case == "Lower":
        sentence_list = [
            [word.lower() for word in token_list] for token_list in sentence_list
        ]
    elif case == "Upper":
        sentence_list = [
            [word.upper() for word in token_list] for token_list in sentence_list
        ]
    else:
        pass
    return sentence_list


def token_restructuring(sentence_list: list):
    """Reduce a nested list of tokens to a single list (1D).

    Args:
        sentence_list (list): list to be work on.
    """
    return reduce(lambda x, y: x + y, sentence_list)

## Aplicação dos filtros

Baixando stop words do idioma especificado e a wordnet para lemmatizaton

In [8]:
import nltk
if remove_stop_words:
    # Download stopwords from nltk
    nltk.download('stopwords')

    # Get a list of stopwords for the defined language
    stopwords = nltk.corpus.stopwords.words(language)

if lemmatization:
    nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Aplicando pré processamento

In [9]:
vocab = tokenize_text(X)
top_tokens = top_tokens_stopwords(vocab) if remove_top_words else None
vocab = remove_specific_tokens(vocab,top_tokens) if remove_top_words else vocab
vocab = remove_specific_tokens(vocab,stopwords) if remove_stop_words else vocab
vocab = apply_stemming(vocab) if stemming else vocab
vocab = apply_lemmatization(vocab) if lemmatization else vocab
vocab = apply_casing(vocab,case)
text = [' '.join(tokens) for tokens in vocab]

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [10]:
y_dec = list(label_encoder.inverse_transform(y))
new_columns = ['review', 'sentiment']
df_result = pd.DataFrame(list(zip(text, y_dec)),columns = new_columns) 
#save dataset changes
df_result.to_csv(dataset, index=False)
df_result

,review,sentiment
0,one of the other reviewer ha mentioned that af...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this wa a wonderful way to spend tim...,positive
3,basically there a family where a little boy ja...,negative
4,petter matteis love in the time of money is a ...,positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,bad plot bad dialogue bad acting idiotic direc...,negative
49997,i am a catholic taught in parochial elementary...,negative
49998,im going to have to disagree with the previous...,negative
